<a href="https://colab.research.google.com/github/BJahanyar/Advanced_Python/blob/main/MachineLearning/Assignment_40/Predicting_House_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [389]:
!pip install wandb

In [390]:
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense


In [391]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [392]:
data_train = pd.read_csv('drive/MyDrive/Colab Notebooks/House_Pricing_Prediction/train.csv')
data_train.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [393]:
#print (data_train['BldgType'].unique() , data_train['MSSubClass'].unique()), data_train['MSZoning'].unique() , data_train['LotFrontage'].unique() , data_train['LotArea'].unique() , data_train['Street'].unique()
#print(data_train.columns)

In [394]:
#________general zoning
data_train = data_train.replace(['A','FV', 'C (all)', 'I', 'RH', 'RL', 'RP', 'RM'],[0,1,2,3,4,5,6,7])

#________Type of road
data_train = data_train.replace(['Pave','Grvl'],[0,1])

#________utilities available
data_train = data_train.replace(['AllPub','NoSewr', 'NoSeWa', 'ELO'],[0,1,2,3])

#________Garage location
data_train = data_train.replace(['2Types','Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd'],[0,1,2,3,4,5])

#________Year Sold
#data_train = data_train.replace(['2006','2007','2008','2009','2010'] ,[0,1,2,3,4])

#________deductions
data_train = data_train.replace(['Typ','Min1', 'Min2', 'Mod', 'Maj1', 'Maj2','Sev', 'Sal'],[0,1,2,3,4,5,6,7])

#________Building Type
data_train = data_train.replace(['1Fam','2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'],[0,1,2,3,4])

data_train = data_train.fillna(0)
data_train.head(2)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,5,65.0,8450,0,0.0,Reg,Lvl,0,Inside,Gtl,CollgCr,Norm,Norm,0,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,0,0,0,1.0,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,5,80.0,9600,0,0.0,Reg,Lvl,0,FR2,Gtl,Veenker,Feedr,Norm,0,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,0,1,TA,1.0,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,0,0,0,5,2007,WD,Normal,181500


In [395]:
X_data= np.array(data_train [['MSZoning','Street' ,'Utilities'  ,'BldgType'  ,'GarageType'  ,'Functional' , 'YrSold',
                               'GarageYrBlt','Fireplaces', 'YearBuilt', 'FullBath','GrLivArea','1stFlrSF','YearRemodAdd',
                               'WoodDeckSF','OpenPorchSF','2ndFlrSF','BsmtUnfSF','GarageArea' ,'3SsnPorch','MSSubClass','TotalBsmtSF',
                               'BedroomAbvGr','WoodDeckSF']])
Y_data= np.array(data_train [['SalePrice']])

X_train, X_validation, Y_train, Y_validation = train_test_split(X_data, Y_data, test_size=0.2)


print('X_train.shape --------->',X_train.shape)
#print("X_train --------------->",X_train)
print('X_validation.shape ---->',X_validation.shape)
#print("X_validation ---------->",X_validation)
print('Y_train.shape --------->',Y_train.shape)
#print("Y_train --------------->",Y_train)
print('Y_validation.shape ---->',Y_validation.shape)
#print("Y_validation----------->",Y_validation)





X_train.shape ---------> (1168, 24)
X_validation.shape ----> (292, 24)
Y_train.shape ---------> (1168, 1)
Y_validation.shape ----> (292, 1)


In [396]:
model = tf.keras.models.Sequential([
      Dense(32 , activation = "relu"),
      Dense(64, activation='relu'),
      Dense(128, activation='relu'),
      Dense(256, activation='relu'),
      Dense(128, activation='relu'),
      Dense(64, activation='relu'),
      Dense(32, activation='relu'),
      Dense(1, activation='relu'),
])

In [397]:
wandb.init(project='Predicting_House_Price', entity='baharjahanyar')

epoch,399
loss,5.15802
val_loss,13.13898
_runtime,430
_timestamp,1630762204
_step,1299
best_val_loss,12.13461
best_epoch,107


epoch,▁▁▂▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇▇██▁▁▂▂▂▃▃▃▃▄▄▄
loss,█▇▆▆▅▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁
val_loss,█▇▅▃▃▂▂▂▂▂▂▂▂▃▃▃▃▄▂▂▂▂▂▂▃▂▂▁▁▃▁▃▁▃▁▃▂▂▃▄
_runtime,▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▇▇▇▇▇▇▇█████
_timestamp,▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▇▇▇▇▇▇▇█████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [398]:
config = wandb.config
config.learning_rate = 0.001

In [399]:
model.compile(loss=tf.keras.losses.mean_absolute_percentage_error,
              optimizer=tf.keras.optimizers.Adam(learning_rate = config.learning_rate))

In [406]:
output = model.fit(X_train, Y_train,
                   validation_data=(X_validation, Y_validation),
                   callbacks=[WandbCallback()],
                   epochs =400 )

Epoch 1/400
37/37 [==============================] - 0s 6ms/step - loss: 8.4869 - val_loss: 14.2567
Epoch 2/400
37/37 [==============================] - 0s 4ms/step - loss: 8.1881 - val_loss: 14.1436
Epoch 3/400
37/37 [==============================] - 0s 5ms/step - loss: 8.5861 - val_loss: 14.5826
Epoch 4/400
37/37 [==============================] - 0s 4ms/step - loss: 7.9481 - val_loss: 15.7962
Epoch 5/400
37/37 [==============================] - 0s 5ms/step - loss: 8.2370 - val_loss: 14.5356
Epoch 6/400
37/37 [==============================] - 0s 5ms/step - loss: 8.2067 - val_loss: 14.5308
Epoch 7/400
37/37 [==============================] - 0s 4ms/step - loss: 8.6882 - val_loss: 14.3903
Epoch 8/400
37/37 [==============================] - 0s 4ms/step - loss: 9.0779 - val_loss: 14.4955
Epoch 9/400
37/37 [==============================] - 0s 4ms/step - loss: 8.7654 - val_loss: 14.4836
Epoch 10/400
37/37 [==============================] - 0s 4ms/step - loss: 8.7512 - val_loss: 14.6030

In [407]:
data_test = pd.read_csv('drive/MyDrive/Colab Notebooks/House_Pricing_Prediction/test.csv')

In [408]:
#________general zoning
data_test = data_test.replace(['A','FV', 'C (all)', 'I', 'RH', 'RL', 'RP', 'RM'],[0,1,2,3,4,5,6,7])

#________Type of road
data_test = data_test.replace(['Pave','Grvl'],[0,1])

#________utilities available
data_test = data_test.replace(['AllPub','NoSewr', 'NoSeWa', 'ELO'],[0,1,2,3])

#________Type of dwelling
data_test = data_test.replace(['1Fam','2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'],[0,1,2,3,4])

#________Garage location
data_test = data_test.replace(['2Types','Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd'],[0,1,2,3,4,5])

#________deductions
data_test = data_test.replace(['Typ','Min1', 'Min2', 'Mod', 'Maj1', 'Maj2','Sev', 'Sal'],[0,1,2,3,4,5,6,7])

data_test = data_test.fillna(0)
data_test.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,4.0,80.0,11622,0,0.0,Reg,Lvl,0.0,Inside,Gtl,NAmes,Feedr,Norm,0,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,0.0,0,0,1.0,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,0,MnPrv,0,0,6,2010,WD,Normal
1,1462,20,5.0,81.0,14267,0,0.0,IR1,Lvl,0.0,Corner,Gtl,NAmes,Norm,Norm,0,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,0.0,0,0,1.0,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,0,0,Gar2,12500,6,2010,WD,Normal


In [409]:
Y_data= pd.read_csv('drive/MyDrive/Colab Notebooks/House_Pricing_Prediction/sample_submission.csv')

X_test= np.array(data_test [['MSZoning','Street' ,'Utilities'  ,'BldgType'  ,'GarageType'  ,'Functional' , 'YrSold',
                               'GarageYrBlt','Fireplaces', 'YearBuilt', 'FullBath','GrLivArea','1stFlrSF','YearRemodAdd',
                               'WoodDeckSF','OpenPorchSF','2ndFlrSF','BsmtUnfSF','GarageArea' ,'3SsnPorch','MSSubClass','TotalBsmtSF',
                               'BedroomAbvGr','WoodDeckSF']])
Y_test = np.array(Y_data[['SalePrice']])

print("X_test.shape ----> ", X_test.shape)
#print("X_test----------->",  X_test)
print("Y_test.shape ----> ", Y_test.shape)
#print("Y_test----------->",  Y_test)


X_test.shape ---->  (1459, 24)
Y_test.shape ---->  (1459, 1)


In [410]:
model.evaluate(X_test,Y_test)

46/46 [==============================] - 0s 2ms/step - loss: 28.6454


28.64537239074707